In [1]:
# Import necessary tools
import tensorflow as tf
import tensorflow_hub as hub 
print("TF version:", tf.__version__)
print("TF Hub version:", hub.__version__)

# Check for GPU availability
print("GPU", "available (YESSSS!!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

TF version: 2.9.2
TF Hub version: 0.12.0
GPU available (YESSSS!!!!!)


## Getting our data ready (turning into Tensors)

With all machine learning models, our data has to be in numerical format. So that's what we'll be doing first. Turning our images into Tensors (numerical representations).

Let's start by accessing our data and checking out the labels.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DATA_PATH = "drive/MyDrive/Colab Notebooks/data/dog-vision"
# "drive/My Drive/Dog Vision"

In [4]:
# Checkout the labels of our data
import pandas as pd
labels_csv = pd.read_csv(f"{DATA_PATH}/labels.csv")

### Getting images and their labels 

Let's get a list of all of our image file pathnames.

In [5]:
# Create pathnames from imageID's
filenames = [f"{DATA_PATH}/train/{fname}.jpg" for fname in labels_csv["id"]]

In [6]:
# Check weather number of filenames matches number of actual image files
import os
if len(os.listdir(f"{DATA_PATH}/train/")) == len(filenames):
  print("Filenames match actual amount of files!!! Proceed.")
else:
  print("Filenames do no match actual amount of files, check the target directory.")

Filenames match actual amount of files!!! Proceed.


Since we've now got our training image filepaths in a list, let's prepare our labels

In [7]:
import numpy as np
labels = labels_csv["breed"].to_numpy() 
# labels = np.array(labels) # does same thing as above
labels

array(['boston_bull', 'dingo', 'pekinese', ..., 'airedale',
       'miniature_pinscher', 'chesapeake_bay_retriever'], dtype=object)

In [8]:
# See if number of labels matches the number of filenames
if len(labels) == len(filenames):
  print("Number of labels matches number of filenames!")
else:
  print("Number of labels does not match number of filenames, check data directories!")

Number of labels matches number of filenames!


In [9]:
# Find the unique label values
unique_breeds = np.unique(labels)

# Turn every label into a boolean array
boolean_labels = [label == unique_breeds for label in labels]

### Creating our own validation set
Since the dataset from Kaggle doesn't come with a validation set, we're going to create our own.

In [10]:
# Setup X & y variables
X = filenames
y = boolean_labels

We're going to start off experimenting with ~1000 images and increase as needed.

In [11]:
# Set number of images to use for experimenting
NUM_IMAGES = 1000 #@param {type:"slider", min:1000, max:10000, step:1000}

In [12]:
# Let's split our data into train and validation sets
from sklearn.model_selection import train_test_split

# Split them into training and validation of total size NUM_IMAGES
X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES],
                                                  y[:NUM_IMAGES],
                                                  test_size=0.2,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

(800, 800, 200, 200)

## Preprocessing Images (turning images into Tensors)

To preprocess our images into Tensors we're going to write a function which does a few things:
1. Take an image filepath as input
2. Use TensorFlow to read the file and save it to a variable, `image`
3. Turn our `image` (a jpg) into Tensors
4. Normalize our image (convert color channel values from 0-255 to 0-1).
5. Resize the `image` to be a shape of (224, 224)
6. Return the modified `image`

Before we do, let's see what importing an image looks like.

In [13]:
# Define image size
IMG_SIZE = 224

# Create a function for preprocessing images
def process_image(image_path, img_size=IMG_SIZE):
  """
  Takes an image file path and turns the image into a Tensor.
  """
  # Read in an image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired value (224, 224)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])

  return image

## Turning our data into batches

Why turn our data into batches?

Let's say you're trying to process 10,000+ images in one go... they all might not fit into memory.

So that's why we do about 32 (this is the batch size) images at a time (you can manually adjust the batch size if need be).

In order to use TensorFlow effectively, we need our data in the form of Tensor tuples which look like this: 
`(image, label)`.

In [14]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the assosciated label,
  processes the image and reutrns a typle of (image, label).
  """
  image = process_image(image_path)
  return image, label

Now we've got a way to turn our data into tuples of Tensors in the form: `(image, label)`, let's make a function to turn all of our data (`X` & `y`) into batches!

In [15]:
# Define the batch size, 32 is a good start
BATCH_SIZE = 32

# Create a function to turn data into batches
def create_data_batches(X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (X) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle if it's validation data.
  Also accepts test data as input (no labels).
  """
  # If the data is a test dataset, we probably don't have have labels
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) # only filepaths (no labels)
    data_batch = data.map(process_image).batch(batch_size)
  
  # If the data is a valid dataset, we don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.map(get_image_label).batch(batch_size)

  else:
    print("Creating training data batches...")
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(X))

    # Create (image, label) tuples (this also turns the iamge path into a preprocessed image)
    data = data.map(get_image_label)

    # Turn the training data into batches
    data_batch = data.batch(batch_size)
  return data_batch

In [16]:
# Create training and validation data batches
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

Creating training data batches...
Creating validation data batches...


In [17]:
# Create a function to load a trained model
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path, 
                                     custom_objects={"KerasLayer":hub.KerasLayer})
  return model

In [18]:
# Load a trained model
loaded_1000_image_model = load_model(f"{DATA_PATH}/models/20230130-05511675057901-1000-images-mobilenetv2-Adam.h5")

Loading saved model from: drive/MyDrive/Colab Notebooks/data/dog-vision/models/20230130-05511675057901-1000-images-mobilenetv2-Adam.h5


---

## Making and evaluating predictions using a trained model 

In [19]:
model = loaded_1000_image_model

In [20]:
val_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 120), dtype=tf.bool, name=None))>

In [21]:
predictions = model.predict(val_data, verbose=1)
predictions

7/7 [==============================] - 13s 1s/step


array([[1.25392037e-03, 3.34030556e-05, 5.87509596e-04, ...,
        4.73118744e-05, 3.62558217e-06, 1.43094501e-03],
       [2.90242443e-03, 1.85185601e-03, 3.75254676e-02, ...,
        7.77256500e-05, 6.68112305e-04, 3.13210214e-04],
       [6.89202716e-05, 3.23318673e-05, 2.52661757e-05, ...,
        3.24379871e-05, 4.57434107e-05, 1.91236992e-04],
       ...,
       [7.44460579e-07, 1.24601011e-05, 5.27296324e-05, ...,
        1.43923189e-05, 1.61705211e-05, 4.21006334e-05],
       [7.96642900e-03, 1.06169260e-04, 2.29836616e-04, ...,
        1.07955406e-04, 4.51440610e-05, 7.18181999e-03],
       [5.76020975e-04, 7.70392944e-05, 5.66613744e-04, ...,
        1.94099580e-03, 2.56530242e-03, 5.85808011e-05]], dtype=float32)

In [22]:
predictions[0]  # using softmax activation

array([1.25392037e-03, 3.34030556e-05, 5.87509596e-04, 1.82466698e-04,
       6.18016697e-04, 6.77973730e-05, 3.94250713e-02, 2.50931684e-04,
       3.89839028e-04, 8.64737725e-04, 1.36085815e-04, 2.70503078e-05,
       2.94261728e-04, 1.57569521e-05, 8.80287902e-04, 8.18714558e-04,
       4.09523564e-05, 3.53939623e-01, 7.61584488e-06, 9.06319401e-05,
       1.26225292e-04, 1.64255121e-04, 1.12546941e-05, 2.31786445e-03,
       2.43164704e-05, 1.80907271e-04, 2.04054102e-01, 9.09653245e-05,
       9.69130197e-05, 2.64032104e-04, 5.34940162e-04, 1.17940817e-03,
       2.89182906e-04, 4.68568760e-05, 1.32750196e-04, 1.40508693e-02,
       7.99532427e-06, 1.09747157e-03, 7.62372511e-05, 1.36382863e-04,
       6.33108080e-04, 4.92421213e-06, 7.05859784e-05, 2.58097745e-04,
       1.80031748e-05, 1.86109814e-04, 7.97800749e-05, 2.48167253e-05,
       1.57213799e-04, 2.18986344e-04, 1.24590879e-04, 2.67514351e-05,
       4.96608147e-04, 1.79257713e-05, 1.24567423e-05, 1.97273021e-05,
      

In [23]:
len(predictions[0])

120

In [24]:
predictions.shape

(200, 120)

In [25]:
np.sum(predictions[0])

1.0000002

In [26]:
np.sum(predictions[1])

1.0000001

In [27]:
# First prediction
index = 42
print(predictions[index])
print(f"Max value (probability of prediction): {np.max(predictions[index])}")
print(f"Sum: {np.sum(predictions[index])}")
print(f"Max index: {np.argmax(predictions[index])}")
print(f"Predicted label: {unique_breeds[np.argmax(predictions[index])]}")

[3.50392584e-05 5.86264105e-05 1.45507147e-05 7.16192471e-06
 5.62532572e-04 8.74987109e-06 4.55970294e-05 4.61054151e-04
 3.45369126e-03 1.49791269e-02 4.32126762e-06 1.35657262e-06
 3.27824615e-04 2.08758214e-03 3.85656342e-04 4.11873334e-04
 2.30520272e-05 5.01037612e-05 5.74141704e-05 1.00697966e-04
 1.34415950e-06 2.44295341e-04 8.91110267e-06 1.60881355e-05
 2.86730542e-03 1.93116157e-05 1.02197282e-05 3.05112590e-05
 8.63541791e-05 6.24165614e-06 1.20076775e-05 7.12570400e-05
 1.74812176e-05 7.23716175e-06 1.90639075e-05 2.43896939e-05
 8.21923750e-05 9.81947087e-05 1.31685247e-05 3.91808003e-02
 1.79603339e-05 6.53347524e-06 2.78224726e-03 1.07864196e-06
 4.48248284e-05 2.81346511e-05 1.71700813e-05 2.69971468e-04
 3.08141753e-05 1.35936221e-04 1.44374517e-05 1.17084974e-05
 1.93901680e-04 2.24136957e-03 3.78847585e-06 2.59254943e-04
 8.34026796e-05 3.61426974e-05 4.45251826e-05 6.78372999e-06
 1.14997338e-05 4.33694222e-04 1.15528201e-06 6.63887186e-05
 1.14162167e-05 3.606488

In [28]:
# Second prediction
index = 0
print(f"Max value (probability of prediction): {np.max(predictions[index])}")
print(f"Sum: {np.sum(predictions[index])}")
print(f"Max index: {np.argmax(predictions[index])}")
print(f"Predicted label: {unique_breeds[np.argmax(predictions[index])]}")

Max value (probability of prediction): 0.35393962264060974
Sum: 1.000000238418579
Max index: 17
Predicted label: border_terrier


Having the above functionality is great but we want to be able to do it at scale.

And it would be even better if we could see the image the prediction is being made on!

**Note:** Prediction probabilities are also known as confidence levels.

In [29]:
# Turn prediction probabilities into their respective label (easier to understand)
def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return unique_breeds[np.argmax(prediction_probabilities)]

# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[81])
pred_label

'brittany_spaniel'

Now since our validation data is still in a batch dataset, we'll have to unbatchify it to make predictions on the validation images and then compare those predictions to the validation labels (truth labels).

In [30]:
val_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 120), dtype=tf.bool, name=None))>

In [31]:
# Create a function to unbatch a batch dataset
def unbatchify(data):
  """
  Takes a batched dataset of (image, label) Tensors and reutrns separate arrays
  of images and labels.
  """
  images = []
  labels = []
  # Loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(label)
  return images, labels

# Unbatchify the validation data
val_images, val_labels = unbatchify(val_data)
val_images[0], val_labels[0]

(array([[[0.29599646, 0.43284872, 0.3056691 ],
         [0.26635826, 0.32996926, 0.22846507],
         [0.31428418, 0.27701408, 0.22934894],
         ...,
         [0.77614343, 0.82320225, 0.8101595 ],
         [0.81291157, 0.8285351 , 0.8406944 ],
         [0.8209297 , 0.8263737 , 0.8423668 ]],
 
        [[0.2344871 , 0.31603682, 0.19543913],
         [0.3414841 , 0.36560842, 0.27241898],
         [0.45016077, 0.40117094, 0.33964607],
         ...,
         [0.7663987 , 0.8134138 , 0.81350833],
         [0.7304248 , 0.75012016, 0.76590735],
         [0.74518913, 0.76002574, 0.7830809 ]],
 
        [[0.30157745, 0.3082587 , 0.21018331],
         [0.2905954 , 0.27066195, 0.18401104],
         [0.4138316 , 0.36170745, 0.2964005 ],
         ...,
         [0.79871625, 0.8418535 , 0.8606443 ],
         [0.7957738 , 0.82859945, 0.8605655 ],
         [0.75181633, 0.77904975, 0.8155256 ]],
 
        ...,
 
        [[0.9746779 , 0.9878955 , 0.9342279 ],
         [0.99153054, 0.99772066, 0.94278

In [32]:
get_pred_label(val_labels[0])

'cairn'